In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0].split('BecaNLP')[0],'BecaNLP/Utils'))

import NLPUtils as nlp

%load_ext autoreload
%autoreload 2

In [36]:
from NLPUtils.datasets import imdb
from sklearn.feature_extraction.text import CountVectorizer
from NLPUtils.datasets.utils import NgramTextVectorizer
import numpy as np

train_corpus = ['Esto es una prueba',
          'esto también',
          'Esto es una prueba más, y esto también',
          'esto es la última prueba.']

dev_corpus = ['Esto es una prueba para dev',
              'esto también',
              'esto ya no importa pero igual']

token_pattern = r'\b\w+\b'
ngram_range = (1,1)
vocabulary = None #['esto', 'es', 'prueba','igual']
unk_token = None
vec1 = NgramTextVectorizer(token_pattern=token_pattern,ngram_range=ngram_range,vocabulary=vocabulary,unk_token=unk_token)
vec2 = CountVectorizer(lowercase=False,token_pattern=token_pattern,ngram_range=ngram_range,vocabulary=vocabulary)

vec1.fit(train_corpus)
vec2.fit(train_corpus)

{0: 1, 1: 1, 2: 1, 3: 1}
{'Esto': 0, 'es': 1, 'una': 2, 'prueba': 3}
{4: 1, 5: 1}
{'Esto': 0, 'es': 1, 'una': 2, 'prueba': 3, 'esto': 4, 'también': 5}
{0: 1, 1: 1, 2: 1, 3: 1, 6: 1, 7: 1, 4: 1, 5: 1}
{'Esto': 0, 'es': 1, 'una': 2, 'prueba': 3, 'esto': 4, 'también': 5, 'más': 6, 'y': 7}
{4: 1, 1: 1, 8: 1, 9: 1, 3: 1}
{'Esto': 0, 'es': 1, 'una': 2, 'prueba': 3, 'esto': 4, 'también': 5, 'más': 6, 'y': 7, 'la': 8, 'última': 9}


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\b\\w+\\b', tokenizer=None,
                vocabulary=None)

In [ ]:
print(vec1.vocabulary)
print()
print(vec2.vocabulary_)

In [30]:
X1 = vec1.transform(dev_corpus)
X2 = vec2.transform(dev_corpus)
indeces = [vec2.vocabulary_[tk if isinstance(tk,str) else ' '.join(tk)] for tk in vec1.vocabulary]
X2 = X2[:,indeces]

print(X1.toarray())
print(X2.toarray())
print(np.all(X1 != X2).sum() == 0)

{'esto': 0, 'es': 1, 'prueba': 2, 'igual': 3, None: 4}
{'esto': 0, 'es': 1, 'prueba': 2}

{'esto': 0, 'es': 1, 'prueba': 2, 'igual': 3}
[[0 1 1]
 [1 0 0]
 [1 0 0]]
[[0 1 1]
 [1 0 0]
 [1 0 0]]
True


# Todas las posibilidades:

## Vocabulario abierto, unk_token = UNK

En train:
```
inicializo el vocabulario en cero
Para cada texto:
    tokenizo ngramas. Para cada token:
        si pertenece al vocabulario, obtengo el índice.
        si no, le asigno un nuevo índice y lo agrego al final del vocabulario
    agrego la cuenta de todos los tokens a la fila de la matriz
agrego un token UNK (columna de ceros en la matriz y último índice en el vocab)
me fijo qué columnas suman entre max_freq y min_freq, las elimino y sumo su contenido a columna UNK
```

En dev:
```
para cada texto:
    tokenizo ngramas. Para cada token:
        si pertenece al vocabulario devuelvo vocab[tk]
        si no, devuelvo vocab[unk_token]
    agrego la cuenta de todos los tokens a la fila de la matriz        
```

## Vocabulario cerrado, unk_token = UNK

En train:
```
inicializo el vocabulario en cero
Para cada texto:
    tokenizo ngramas. Para cada token:
        si pertenece al vocabulario, obtengo el índice.
        si no, le asigno un nuevo índice y lo agrego al final del vocabulario
    agrego la cuenta de todos los tokens a la fila de la matriz
agrego un token UNK (columna de ceros en la matriz y último índice en el vocab)
me fijo qué columnas suman entre max_freq y min_freq, las elimino y sumo su contenido a columna UNK
también me fijo qué columnas de las que quedaron son palabras del vocabulario, y las que no los son también las elimino y las sumo a la columna UNK
```

En dev:
```
para cada texto:
    tokenizo ngramas. Para cada token:
        si pertenece al vocabulario devuelvo vocab[tk]
        si no, devuelvo vocab[unk_token]
    agrego la cuenta de todos los tokens a la fila de la matriz        
```




In [89]:
from scipy.sparse import csr_matrix

train_corpus = ['Esto es una prueba',
          'esto también',
          'Esto es una prueba más, y esto también',
          'esto es la última prueba.']

vec = CountVectorizer(lowercase=False,token_pattern=token_pattern,ngram_range=(1,1),vocabulary=None)

X = vec.fit_transform(train_corpus)
X = csr_matrix((X.data,X.indices,X.indptr),shape=(X.shape[0],X.shape[1]+1))
vocab = vec.vocabulary_
vocab['UNK'] = len(vocab)
print(X.toarray())
print(vocab)

def get_columns_to_remove(X,min_freq,max_freq):
    sums = np.asarray(X.sum(axis=0)).reshape(-1)
    x1 = sums >= min_freq
    x2 = sums <= max_freq
    ands = np.logical_not(np.logical_and(x1,x2))
    remove_items = list(np.where(ands)[0])
    return remove_items

def remove_tokens(vocab,X,remove_tokens,unk_idx=-1):
    """
    elimina los tokens de un vocabulario ordenado 
    """
    X[:,unk_idx] = X[:,remove_tokens].sum(axis=1)
    true_items = sorted(list(set(range(len(vocab))) - set(remove_tokens)))
    X = X[:,true_items]
    tokens = list(vocab.keys())
    vocab = {tokens[i]:idx for idx, i in enumerate(true_items)}
    return X, vocab
    
tokens_to_remove = get_columns_to_remove(X,0,2)
print(tokens_to_remove)
X, vocab = remove_tokens(vocab,X,tokens_to_remove,unk_idx=-1)
print(X.toarray())
print(vocab)

[[1 1 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0]
 [1 1 1 0 1 1 1 1 1 0 0]
 [0 1 1 1 0 1 0 0 0 1 0]]
{'Esto': 0, 'es': 1, 'una': 7, 'prueba': 5, 'esto': 2, 'también': 6, 'más': 4, 'y': 8, 'la': 3, 'última': 9, 'UNK': 10}
[1, 2, 5]
[[1 0 0 0 1 0 0 2]
 [0 0 0 1 0 0 0 1]
 [1 0 1 1 1 1 0 3]
 [0 1 0 0 0 0 1 3]]
{'Esto': 0, 'prueba': 1, 'esto': 2, 'más': 3, 'y': 4, 'la': 5, 'última': 6, 'UNK': 7}


/home/lestien/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [166]:
import scipy.sparse as sp

class NgramTextVectorizer(CountVectorizer):
    
    def __init__(self,token_pattern=r'\b\w+\b',vocabulary=None,unk_token=None,
         min_freq=1,max_freq=np.inf,ngram_range=(1,1)):
        
        super().__init__(lowercase=False,token_pattern=token_pattern,vocabulary=None,ngram_range=ngram_range)

        self.unk_token = unk_token
        self.min_freq = min_freq
        self.max_freq = max_freq
        
        if vocabulary is None or isinstance(vocabulary,list) or isinstance(vocabulary,set) or hasattr(vocabulary,'__iter__'):
            self.vocab = vocabulary
        elif isinstance(vocabulary,dict):
            self.vocab = list(vocabulary.keys())
        else:
            raise TypeError('El vocabulario tiene que ser un iterable de palabras')
        
    def fit_transform(self,corpus):
        X = super().fit_transform(corpus)

        vocab = self.vocab
        if vocab is not None:
            n_appended = 0
            for tk in vocab:
                if tk not in self.vocabulary_:
                    self.vocabulary_[tk] = len(self.vocabulary_)
                    n_appended += 1
            X.resize((X.shape[0],X.shape[1]+n_appended))
        
        remove_items = self.get_columns_to_remove(X,self.min_freq,self.max_freq)
        if vocab is not None:
            stay_items = [self.vocabulary_[tk] for tk in self.vocabulary_.keys() if all([sub_tk in vocab for sub_tk in tk.split(' ')])]
            remove_items = np.unique(np.hstack((remove_items,
                  list(set(range(len(self.vocabulary_))) - set(stay_items)))))
        
        unk_token = self.unk_token
        if unk_token is not None:
            self.vocabulary_[unk_token] = len(self.vocabulary_)
            X.resize(X.shape[0],X.shape[1]+1)
        
        X = self.remove_tokens(X,remove_items)
        return X
        
    def transform(self,corpus):
        
        unk_token = self.unk_token
        vocab = self.vocabulary_
        
        indptr = [0]
        j_indices = []
        values = []
        
        tokenizer = self.build_analyzer()
        if unk_token is None:
            for doc in corpus:
                feature_counter = {}
                for feature in tokenizer(doc):
                    try:
                        feature_idx = vocab[feature]
                        if feature_idx not in feature_counter:
                            feature_counter[feature_idx] = 1
                        else:
                            feature_counter[feature_idx] += 1
                    except KeyError:
                        pass

                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))
                
        else:
            for doc in corpus:
                feature_counter = {}
                for feature in tokenizer(doc):
                    try:
                        feature_idx = vocab[feature]
                    except KeyError:
                        feature_idx = vocab[unk_token]

                    if feature_idx not in feature_counter:
                        feature_counter[feature_idx] = 1
                    else:
                        feature_counter[feature_idx] += 1

                j_indices.extend(feature_counter.keys())
                values.extend(feature_counter.values())
                indptr.append(len(j_indices))
            
        j_indices = np.asarray(j_indices, dtype=self.dtype)
        indptr = np.asarray(indptr, dtype=self.dtype)
        values = np.asarray(values, dtype=self.dtype)
        
        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocab)),
                          dtype=self.dtype)
        X.sort_indices()
            
        return X
        
    def fit(self,corpus):
        self.fit_transform(corpus)
        return self
        
    def get_columns_to_remove(self,X,min_freq,max_freq):
        sums = np.asarray(X.sum(axis=0)).reshape(-1)
        x1 = sums >= min_freq
        x2 = sums <= max_freq
        ands = np.logical_not(np.logical_and(x1,x2))
        remove_items = np.where(ands)[0]
        return remove_items

    def remove_tokens(self,X,remove_tokens):
        if self.unk_token is not None:
            X = X.tolil()
            X[:,-1] = X[:,remove_tokens].sum(axis=1)
            X = X.tocsr()
        true_items = sorted(list(set(range(len(self.vocabulary_))) - set(remove_tokens)))
        X = X[:,true_items]
        
        sorted_idx = np.argsort(list(self.vocabulary_.values()))
        tokens = list(self.vocabulary_.keys())
        tokens = [tokens[i] for i in sorted_idx]
        self.vocabulary_ = {tokens[i]:idx for idx, i in enumerate(true_items)}
        return X

train_corpus = ['Esto es una prueba',
          'esto también',
          'Esto es una prueba más, y esto también',
          'esto es la última prueba.']

dev_corpus = ['Esto es una prueba para dev',
              'esto también',
              'esto ya no importa pero igual']

token_pattern = r'\b\w+\b'
ngram_range = (1,2)
vocabulary = ['esto', 'es', 'prueba','igual']
unk_token = None#'UNK'
vec1 = NgramTextVectorizer(token_pattern=token_pattern,ngram_range=ngram_range,
                           vocabulary=vocabulary,unk_token=unk_token,min_freq=0,max_freq=np.inf)

vec1.fit(train_corpus)
print(vec1.vocabulary_)
X = vec1.transform(dev_corpus)
X.toarray()

#print(vec1.vocabulary_)

{'es': 0, 'esto': 1, 'esto es': 2, 'prueba': 3, 'igual': 4}


array([[1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1]])

In [169]:
train_corpus = imdb.train_reader()['comment']
dev_corpus = imdb.test_reader()['comment'].sample(n=5000,replace=False,random_state=0)

token_pattern = r'\b\w+\b'
ngram_range = (1,2)
vocabulary = None#['esto', 'es', 'prueba','igual']
unk_token = 'UNK'
vec1 = NgramTextVectorizer(token_pattern=token_pattern,ngram_range=ngram_range,
                           vocabulary=vocabulary,unk_token=unk_token,min_freq=0,max_freq=np.inf)

tic = time.time()
print('start')
vec1.fit(train_corpus)
toc1 = time.time()
print(toc1-tic)
X = vec1.transform(dev_corpus)
toc2 = time.time()
print(toc2-tic)

start
70.49143838882446
77.25798797607422
